In [10]:
import pandas as pd
import numpy as np
import random
import allel
import h5py
random.seed(42)
import time
np.random.seed(42)

In [11]:
callset_fn = '/kwiat/vector/ag1000g/release/phase2.AR1/variation/main/hdf5/biallelic/ag1000g.phase2.ar1.pass.biallelic.3L.h5'
callset = h5py.File(callset_fn, mode='r')

# #get samples
# df_samples = pd.read_csv('/kwiat/vector/ag1000g/release/phase2.AR1/samples/tbl_samples.txt', delimiter='\t')
# all_samp=list(df_samples.ox_code)

regions = (("3R", 1000000, 37000000, "free"),
           ("3L", 15000000, 41000000, "free"))

chrom = '3L'

#get genotypes
g = allel.GenotypeChunkedArray(callset[chrom]['calldata']['genotype'])

#get positions
p = allel.VariantChunkedTable(callset[chrom]['variants'],
                                 names=['POS'],
                                 index='POS')
pos = p.POS[:]

#just keep positions within the free region
pos_free = pos[(pos > 15000000) * (pos < 41000000)] 
free_b = p.index.locate_keys(pos_free)
g_snp = g.compress(free_b, axis=0)

#filter - no missing data
ac = g_snp.count_alleles()
flt = (ac.max_allele() == 1) & (ac[:, :2].min(axis=1) > 1)
gf = g_snp.compress(flt, axis=0)

#turn the genotype array into 2D matrix of non_ref alleles per call
gn = gf.to_n_alt()

#randomly select snps
n = 100000
vidx = np.random.choice(gn.shape[0], n, replace=False)
vidx.sort()
gnr = gn.take(vidx, axis=0)

In [14]:
#LD prune
def ld_prune(gn, size, step, threshold=.1, n_iter=1):
    for i in range(n_iter):
        loc_unlinked = allel.stats.loc_unlinked(gn, size=size, step=step, threshold=threshold)
        n = np.count_nonzero(loc_unlinked)
        n_remove = gn.shape[0] - n
        print('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
        gn = gn.compress(loc_unlinked, axis=0)
    return gn

In [18]:
from allel.stats.window import windowed_statistic
from allel.util import asarray_ndim, ensure_square
from allel.chunked import get_blen_array
from allel.compat import memoryview_safe
from allel.opt.stats import gn_pairwise_corrcoef_int8, gn_pairwise2_corrcoef_int8, gn_locate_unlinked_int

ImportError: cannot import name 'gn_locate_unlinked_int'

In [15]:
gnu = ld_prune(gnr, size=500, step=200, threshold=.1, n_iter=10)

AttributeError: module 'allel.stats' has no attribute 'loc_unlinked'

In [ ]:
allel.s